### Predict biomass from extracted HLS indices/bands using a saved model

In [1]:
import pandas as pd
import pickle
import importlib.resources as pkg_resources
from src.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
from src import models

In [2]:
pkg_resources.read_text(models, 'CPER_HLS_to_VOR_biomass_model_lr_simp.pk')
#mod_bm = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk'

In [3]:
import pkgutil
pkgutil.get_data("src", "models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk")

FileNotFoundError: [Errno 2] No such file or directory: '/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk'